In [ ]:
pip install openclean

In [ ]:
pip install humanfriendly

In [33]:
from openclean.data.source.socrata import Socrata

for dataset in Socrata().catalog(domain='data.cityofnewyork.us'):
    if 'parking' in dataset.name.lower() and 'violation' in dataset.name.lower():
        print(f'{dataset.identifier}\t{dataset.domain}\t{dataset.name}')

nc67-uf89	data.cityofnewyork.us	Open Parking and Camera Violations
pvqr-7yc4	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2023
ncbg-6agr	data.cityofnewyork.us	DOF Parking Violation Codes
jt7v-77mi	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2014
kiv2-tbus	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2016
2bnn-yakx	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2017
faiq-9dfq	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2019
a5td-mswe	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2018
c284-tqph	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2015
p7t3-5i9s	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2020
kvfd-bves	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2021
7mxj-7a6y	data.cityofnewyork.us	Parking Violations Issued - Fiscal Year 2022


In [34]:
import gzip
import humanfriendly
import os

dataset = Socrata().dataset('jt7v-77mi')

# By default, this example uses a small sample of the full dataset that
# is included in the 'data' subfolder within this repository.
#datafile = './data/jt7v-77mi.tsv.gz'

# Remove the comment for this line if you want to use the full dataset.
datafile = 'jt7v-77mi.tsv.gz'


# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)


fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))


Using 'Parking Violations Issued - Fiscal Year 2014' in file jt7v-77mi.tsv.gz of size 379.19 MB


In [35]:
# Due to the size of the full dataset file, we make use of openclean's
# stream operator to avoid having to load the dataset into main-memory.

from openclean.pipeline import stream

ds_full = stream(datafile)

In [36]:
# Count number of records in the datasets.

print(f'{ds_full.count():,} rows.')

9,100,278 rows.


In [37]:
# Print the first ten rows of the dataset to get a first
# idea of the content.

ds_full.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1361929741,FCJ5493,NY,PAS,12/18/1970,20,SUBN,GMC,S,35030,...,BLACK,0,2013,-,0,,,,,
1,1366962000,63540MC,NY,COM,02/02/1971,46,DELV,FRUEH,P,58830,...,BRN,0,2013,-,0,,,,,
2,1342296187,GCY4187,NY,SRF,09/18/1971,21,VAN,FORD,S,11790,...,BLUE,0,2002,-,0,,,,,
3,1342296199,95V6675,TX,PAS,09/18/1971,21,,GMC,S,11790,...,SILVR,0,2008,-,0,,,,,
4,1342296217,FYM5117,NY,SRF,09/18/1971,21,SUBN,NISSA,S,28190,...,WHITE,0,2012,-,0,,,,,
5,1356906515,GFM1421,NY,PAS,09/18/1971,40,SDN,MAZDA,X,13610,...,BLK,0,2010,-,7,,,,,
6,1337077380,18972BB,NY,999,10/10/1971,14,BUS,INTER,P,8440,...,YELLO,0,2001,-,0,,,,,
7,1364523796,WNJ4730,VA,PAS,04/05/1973,14,SDN,TOYOT,P,50830,...,BLK,0,0,-,0,,,,,
8,1359914924,68091JZ,NY,COM,07/22/1973,46,DELV,TOYOT,P,10610,...,WH,0,2010,-,0,,,,,
9,1355498326,EWV4127,NY,PAS,08/12/1973,21,SUBN,ACURA,X,42630,...,GREY,0,2005,-,0,,,,,


In [38]:
# Create a view on a subset of columns in the dataset.

COLUMNS = [
    'Summons Number',
    'Plate ID',
    'Registration State',
    'Plate Type',
    'Issue Date',
    'Violation Code',
    'Vehicle Body Type',
    'Vehicle Make',
    'Issuing Agency',
    'Vehicle Expiration Date',
    'Violation Time',
    'Violation County',
    'Vehicle Color',
    'Vehicle Year'
]

ds = ds_full.select(columns=COLUMNS)

In [39]:
# Profile the resulting dataset view using the default data profiler.

from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [40]:
# Print overview of profiling results.

profiles.stats()

,total,empty,distinct,uniqueness,entropy
Summons Number,9100278,0,9100278,1.000000,23.117479
Plate ID,9100278,2,2722215,0.299135,20.063750
Registration State,9100278,0,69,0.000008,1.581595
Plate Type,9100278,0,88,0.000010,1.364050
Issue Date,9100278,0,2936,0.000323,8.291448
Violation Code,9100278,0,100,0.000011,4.391310
Vehicle Body Type,9100278,110868,2112,0.000235,2.849453
Vehicle Make,9100278,63175,6873,0.000761,4.774284
Issuing Agency,9100278,0,17,0.000002,1.039213
Vehicle Expiration Date,9100278,0,4415,0.000485,7.972521


In [41]:
# Print the most frequent data type for each column.

print('Schema\n------')
for col in ds.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'Summons Number' (int)
  'Plate ID' (str)
  'Registration State' (str)
  'Plate Type' (str)
  'Issue Date' (date)
  'Violation Code' (int)
  'Vehicle Body Type' (str)
  'Vehicle Make' (str)
  'Issuing Agency' (str)
  'Vehicle Expiration Date' (int)
  'Violation Time' (str)
  'Violation County' (str)
  'Vehicle Color' (str)
  'Vehicle Year' (int)


In [42]:
# Print the minimum and maximum value for column 'Issue Date'

profiles.minmax('Issue Date')

,min,max
date,1970-12-18,2069-12-23


In [43]:
# Print the minimum and maximum value for column 'Vehicle Year'

profiles.minmax('Vehicle Year')

,min,max
int,0,2069


In [44]:
# Print the most frequent values in column 'Violation Time'

profiles.column('Violation Time').get('topValues')

[('0836A', 27495),
 ('1136A', 23682),
 ('0806A', 23229),
 ('0936A', 22780),
 ('1140A', 22323),
 ('0906A', 22291),
 ('0840A', 22215),
 ('0940A', 21360),
 ('0838A', 20529),
 ('0837A', 20365)]

In [45]:
# Get set of distinct values for column 'Registration State'. Print the
# values in decreasing order of frequency.

states = ds.distinct('Registration State')
for rank, val in enumerate(states.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   NY   7,029,804
2   NJ     878,677
3   PA     225,760
4   CT     136,973
5   FL     111,887
6   MA      78,650
7   VA      60,951
8   MD      50,407
9   IN      49,126
10  NC      47,117
11  99      38,080
12  IL      31,763
13  GA      30,837
14  AZ      24,245
15  TX      24,092
16  OH      21,995
17  CA      20,100
18  OK      19,688
19  SC      19,529
20  ME      19,459
21  TN      18,396
22  MI      16,365
23  DE      14,643
24  RI      13,296
25  MN      12,901
26  NH       9,930
27  VT       7,215
28  IA       7,166
29  WA       5,967
30  ID       5,863
31  AL       5,828
32  QB       5,336
33  WI       5,311
34  DP       5,264
35  ON       5,183
36  DC       3,728
37  CO       3,663
38  OR       3,484
39  MS       3,428
40  KY       3,222
41  NM       2,936
42  MO       2,876
43  AR       2,716
44  LA       2,500
45  NV       2,131
46  WV       1,688
47  NE       1,626
48  GV       1,317
49  KS       1,226
50  AK         961
51  UT         942
52  SD         691
53  MT      

In [46]:
# List identifier and names for available reference datasets.

from openclean.data.refdata import RefStore

refdata = RefStore()
for entry in refdata.repository().find():
    print(f'{entry.identifier:<35}:  {entry.name}')

C:\Users\Nikhil\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


company_suffixes                   :  Company Suffixes
encyclopaedia_britannica:us_cities :  Cities in the U.S.
nyc.gov:dof:state_codes            :  NYC Finance - State Codes
restcountries.eu                   :  REST Countries
usps:street_abbrev                 :  C1 Street Suffix Abbreviations
usps:secondary_unit_designators    :  C2 Secondary Unit Designators
wikipedia:us_states                :  States and territories of the U.S.
admins                             :  Geo Administrative Levels


In [47]:
# Download the license plate state codes dataset.

refdata\
    .load('nyc.gov:dof:state_codes', auto_download=True)\
    .df()\
    .head()

,code,name,type
0,AL,Alabama,US
1,AK,Alaska,US
2,AZ,Arizona,US
3,AR,Arkansas,US
4,CA,California,US


In [48]:

# Get set of distinct state codes.

states_ref = refdata.load('nyc.gov:dof:state_codes', auto_download=True).distinct('code')

In [49]:
# Print information for entries in the 'Registration State' column that
# do not occur in the reference dataset.

for rank, val in enumerate(states.most_common()):
    st, freq = val
    if st not in states_ref:
        print(f'{rank + 1:<3} {st}  {freq:>10,}')

11  99      38,080
60  PR         211


Vehicle Expiration Date

In [50]:
# Print the ten most frequent values for the 'Vehicle Expiration Date' column.

expiration_dates = ds.distinct('Vehicle Expiration Date')

for rank, val in enumerate(expiration_dates.most_common(10)):
    dt, freq = val
    print(f'{rank + 1:<3} {dt:>8}  {freq:>10,}')

print('\nTotal number of distinct values is {}'.format(len(expiration_dates)))

1          0   1,036,939
2   88888888   1,034,518
3   88880088     275,925
4   20140088     163,398
5   20130088     155,346
6   20140930     127,904
7   20140430      92,368
8   20141231      91,262
9   20141130      90,542
10  20140228      87,149

Total number of distinct values is 4415


Detect Outliers using scikit-learn

In [51]:
# Using the default settings yields two outliers.

from openclean.profiling.anomalies.sklearn import DBSCANOutliers

DBSCANOutliers().find(expiration_dates)

['0', '88888888']

In [52]:
# If we change the eps parameter (maximum distance between two samples for one to be considered
# as in the neighborhood of the other) we can find even more potential outliers (including one that
# we had not seen before).

DBSCANOutliers(eps=0.05).find(expiration_dates)

['20130088', '0', '88888888', '88880088', '20140088', '19750423']

In [53]:
# Take a look at the record(s) that have an expiration date of '19750423'.

from openclean.function.eval.base import Col

ds_full\
    .filter(Col('Vehicle Expiration Date') == '19750423')\
    .select(['Plate ID', 'Plate Type', 'Registration State', 'Street', 'Vehicle Make', 'Violation Code'])\
    .to_df()

,Plate ID,Plate Type,Registration State,Street,Vehicle Make,Violation Code
631299,GFR1342,PAS,NY,FLUSHING MEADOW CORO,DODGE,20


Data Standardization

In [54]:
# Cluster street names using 'Key Collision' clustering with the
# default fingerprint key generator.

from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

street_names = stream(datafile).update('Street', str.upper).distinct('Street')
clusters = KeyCollision(func=Fingerprint()).clusters(street_names)

In [55]:
# Define simple helper method to print the k largest clusters.

def print_k_clusters(clusters, k=5):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(min(k, len(clusters))):
        print('\nCluster {}'.format(i + 1))
        for key, cnt in clusters[i].items():
            if key == '':
                key = "''"
            print(f'  {key} (x {cnt})')

In [56]:
print_k_clusters(clusters)

Total number of clusters is 4119 with 9164 values

Cluster 1
  W 125 ST (x 3365)
  W 125    ST (x 1)
  W. 125 ST. (x 1)
  W .125 ST (x 5)
  W  125 ST (x 2)
  W 125  ST (x 1)
  W. 125 ST (x 3)

Cluster 2
  FERRY LOT 2 (x 743)
  FERRY LOT #2 (x 140)
  FERRY  LOT #2 (x 1)
  FERRY LOT  2 (x 3)
  FERRY LOT # 2 (x 121)
  FERRY LOT  # 2 (x 2)
  FERRY LOT  #2 (x 1)

Cluster 3
  2ND AVE (x 71826)
  2ND  AVE (x 5)
  2ND AVE. (x 1)
  AVE 2ND (x 1)
  2ND      AVE (x 1)
  2ND    AVE (x 2)
  2ND       AVE (x 1)

Cluster 4
  ST NICHOLAS AVE (x 25913)
  ST. NICHOLAS AVE (x 125)
  ST, NICHOLAS AVE (x 1)
  ST NICHOLAS  AVE (x 9)
  ST NICHOLAS   AVE (x 1)
  ST  NICHOLAS AVE (x 4)
  ST. NICHOLAS  AVE (x 1)

Cluster 5
  LGA TERMINAL B (x 26)
  LGA, TERMINAL B (x 1)
  LGA/ TERMINAL B (x 1)
  TERMINAL B LGA (x 20)
  TERMINAL B - LGA (x 2)
  TERMINAL B -LGA (x 1)
  LGA TERMINAL B, (x 1)


Specialized Key Generators

In [58]:
pip install openclean-geo

     ---------------------------------------- 1.4/1.4 MB 15.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [59]:
# Use a key generator that was specifically designed for street names.

from openclean.cluster.key import KeyCollision
from openclean_geo.address.usstreet import USStreetNameKey

# In this example we take a different approach: we first extract the list of
# distinct street names from the data file. We the apply transformations and
# clustering directly on the list of names using three parallel threads.

clusters = KeyCollision(func=USStreetNameKey(), threads=3).clusters(street_names)
print_k_clusters(clusters, k=5)

Total number of clusters is 10386 with 33342 values

Cluster 1
  W 43 STREET (x 200)
  W 43 ST (x 1666)
  W 43RD ST (x 19864)
  WEST 43 STREET (x 425)
  W 43RD STREET (x 52)
  WEST 43RD ST (x 147)
  WEST 43 ST (x 366)
  WEST 43RD STREET (x 210)
  W 43ST (x 11)
  W 43 RD ST (x 8)
  W 43TH ST (x 10)
  WEST 43RD  STREET (x 1)
  WEST  43 ST (x 1)
  W.43 RD ST (x 1)
  W.43 STREET (x 3)
  W.43RD ST (x 1)
  W.43 ST (x 9)
  WEST 43ST (x 10)
  W43 ST (x 9)
  W. 43 STREET (x 3)
  W43ST (x 1)
  W. 43RD ST (x 1)
  WEST 43TH ST (x 1)
  W .43RD ST (x 2)
  W 43RD  ST (x 1)
  W. 43 ST (x 1)
  W 43 RD STREET (x 1)
  W 43  STREET (x 1)
  W  43 ST (x 1)
  W43RD ST (x 1)
  WEST 43  STREET (x 1)
  W.43 TH  ST (x 1)
  W.43 TH ST (x 1)
  WEST 43TH STREET (x 1)
  WEST 43  ST (x 1)
  W .43 ST (x 1)
  WEST  43ST (x 1)
  WEST 43 RD ST (x 1)

Cluster 2
  W 125 ST (x 3365)
  W 125    ST (x 1)
  W 125 STREET (x 451)
  WEST 125 ST (x 522)
  WEST 125TH ST (x 81)
  W 125TH ST (x 11611)
  WEST 125 STREET (x 354)
  W 12

In [60]:
# Use street name standardization operator to modify street names
# before clustering them using the default fingerprint operator.

from openclean_geo.address.usstreet import StandardizeUSStreetName

street_names_std = StandardizeUSStreetName(characters='upper').apply(street_names, threads=3)
for name, count in street_names_std.most_common(50):
    print(f'{count:6}  {name}')

224649  BROADWAY
171427  3 AVE
110790  5 AVE
 99159  MADISON AVE
 86846  LEXINGTON AVE
 76152  1 AVE
 74313  8 AVE
 74221  2 AVE
 71051  7 AVE
 64495  QUEENS BLVD
 59610  AMSTERDAM AVE
 56776  6 AVE
 54197  JAMAICA AVE
 53699  PARK AVE
 41947  COLUMBUS AVE
 39899  CONEY ISLAND AVE
 38139  37 AVE
 36118  ROOSEVELT AVE
 35986  NOSTRAND AVE
 34006  WHITE PLAINS RD
 33948  HORACE HARDING EXPWY
 33102  9 AVE
 32207  MYRTLE AVE
 31781  ATLANTIC AVE
 31394  MAIN ST
 30573  FLATBUSH AVE
 28925  FULTON ST
 28183  COURT ST
 27882  NORTHERN BLVD
 26273  WEST 58 ST
 26145  ST NICHOLAS AVE
 25806  STEINWAY ST
 25296  WEST 39 ST
 24770  GRAND CONCOURSE
 24455  EAST TREMONT AVE
 24146  JEROME AVE
 24039  13 AVE
 24007  WEST 47 ST
 23423  WB FORDHAM RD @ MARI
 23357  WEST 38 ST
 23326  WEST 35 ST
 22992  WEST 43 ST
 22973  LIBERTY AVE
 22754  WESTCHESTER AVE
 22744  WEST 46 ST
 22650  WEST 48 ST
 22173  10 AVE
 22168  WEST 45 ST
 21600  MANHATTAN AVE
 21358  WEST 37 ST


In [61]:
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(street_names_std)
print_k_clusters(clusters, k=5)

Total number of clusters is 2354 with 5075 values

Cluster 1
  LGA TERMINAL B (x 26)
  LGA , TERMINAL B (x 1)
  LGA / TERMINAL B (x 1)
  TERMINAL B LGA (x 20)
  TERMINAL B - LGA (x 6)
  LGA TERMINAL B , (x 1)

Cluster 2
  B WAY (x 211)
  B - WAY (x 11)
  B / WAY (x 2)
  B . WAY (x 42)
  B . WAY . (x 1)
  B ; WAY (x 1)

Cluster 3
  LGA , CTB (x 1)
  LGA / CTB (x 1)
  LGA CTB (x 10)
  CTB LGA (x 3)
  LGA - CTB (x 1)
  CTB - LGA (x 1)

Cluster 4
  EAST L GRANT HWY (x 48)
  EAST . L GRANT HWY (x 18)
  EAST . L . GRANT HWY (x 25)
  EAST L . GRANT HWY (x 1)
  EAST / L / GRANT HWY (x 1)
  EAST - L GRANT HWY (x 1)

Cluster 5
  JOHN ST (x 4395)
  ST JOHN (x 10)
  ST JOHN ST (x 8)
  ST . JOHN ST (x 1)
  ST . JOHN (x 1)
  JOHN ST . (x 1)


In [62]:
# Use option to remove special characters (keep only alpha-numeric tokens)
# when standardizing street names and option to remove repeated tokens.

f = StandardizeUSStreetName(characters='upper', alphanum=True, repeated=False)
street_names_std = f.apply(street_names, threads=3)
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(street_names_std)
print_k_clusters(clusters, k=5)

Total number of clusters is 761 with 1541 values

Cluster 1
  SOUTH E C O 14 ST (x 1)
  SOUTH E C O E 14 ST (x 1)
  SOUTH O C O E 14 ST (x 1)

Cluster 2
  20 FT FROM C O S W (x 1)
  20 FT FROM S W C O (x 1)
  20 FT FROM S W C O C (x 1)

Cluster 3
  NORTH W C O NORTH 4 ST (x 1)
  NORTH W C O W 4 ST (x 1)
  NORTH W C O 4 ST (x 1)

Cluster 4
  NORTH E C O E 71 (x 5)
  NORTH O C O E 71 (x 1)
  NORTH E C O 71 (x 1)

Cluster 5
  ANN ST (x 1171)
  ST ANN ST (x 1)
  ST ANN (x 2)


Violations of Functional Dependencies

In [63]:
from openclean.function.eval.base import Col
from openclean.function.eval.logic import And
from openclean.function.eval.null import IsNotEmpty

df = ds\
    .select(['Plate ID', 'Registration State', 'Vehicle Color'])\
    .rename('Registration State', 'State')\
    .where(And(IsNotEmpty('Vehicle Color'), Col('State') != '99', Col('Plate ID') != '999'))\
    .update('Vehicle Color', str.upper)\
    .sample(n=100000, random_state=42)\
    .to_df()

In [64]:
# The Plate ID and Registration State should identify a vehicle uniquely. We use
# this key to find conflicts in the 'Vehicle Color' column.

from openclean.operator.map.violations import fd_violations

# Find violations of the FD ['Plate ID', 'Registration State'] -> ['Vehicle Color']

groups = fd_violations(df, lhs=['Plate ID', 'State'], rhs='Vehicle Color')

# Print number of conflicting groups.

print('{} vehicles with conflicting colors'.format(len(groups)))

2626 vehicles with conflicting colors


In [65]:
# Show examples for vehicles that occur in the dataset with
# different colors.

for key in list(groups.keys())[:16:3]:
    print(groups.get(key))
    print('\n')

        Plate ID State Vehicle Color
8785717  57990MB    NY         WHITE
8621712  57990MB    NY            WH
6747726  57990MB    NY         WHITE


        Plate ID State Vehicle Color
4998609  24932MD    NY         WHITE
6302553  24932MD    NY            WH
4280625  24932MD    NY            WH
5553116  24932MD    NY            WH
7443034  24932MD    NY         WHITE
1605393  24932MD    NY            WH
2360021  24932MD    NY            WH
3469710  24932MD    NY            WH


        Plate ID State Vehicle Color
8898869  66734MD    NY            WH
7828016  66734MD    NY         WHITE
3387781  66734MD    NY            WH


        Plate ID State Vehicle Color
6347541  98446JX    NY         BROWN
3652296  98446JX    NY            BR
7053515  98446JX    NY         BROWN
2758740  98446JX    NY         BROWN
3829790  98446JX    NY         BROWN
6599896  98446JX    NY         BROWN


        Plate ID State Vehicle Color
3015441  93788JZ    NY           WHT
8714403  93788JZ    NY        

In [66]:
# Extract the data that is needed for this example. Use only the five
# most common county values and ignore rows with violation times that
# do not match the expected pattern.

from openclean.function.eval.base import Col
from openclean.function.eval.domain import IsIn
from openclean.function.eval.regex import IsMatch

data = ds\
    .filter(IsIn('Violation County', {'K', 'NY', 'Q', 'BX', 'R'}))\
    .filter(IsMatch('Violation Time', pattern=r'\d{4}[AP]', fullmatch=True))\
    .filter(Col('Violation Code') == '21')\
    .rename('Violation County', 'Borough')\
    .select(['Borough', 'Violation Time'])

In [68]:
pip install openclean-notebook

     ---------------------------------------- 1.4/1.4 MB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 56.3/56.3 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [639 lines of output]
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
       -------------------------------------- 807.9/807.9 kB 8.6 MB/s eta 0:00:00
      

In [69]:
pip install openclean-notebook[jupyter]

  Using cached openclean_notebook-0.1.7-py3-none-any.whl (1.4 MB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Using cached datamart_profiler-0.9-py3-none-any.whl (29 kB)
  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [625 lines of output]
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
    Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
    Using cached wheel-0.41

In [70]:
# Create a database object that materializes the dataset and
# all provenance information in a folder on the local file system.

from openclean_notebook import DB

db = DB(basedir='.openclean', create=True)

ModuleNotFoundError: No module named 'openclean_notebook'

In [ ]:
# Load the dataset into the database. Each dataset has to have a
# unique name.

db.load_dataset(data, 'street_cleaning_violations')

In [ ]:
db.stream('street_cleaning_violations').head()

In [ ]:
# Define a mapping of county codes to borough names.

county_map = {
    'K': 'Brooklyn',
    'NY': 'Manhattan',
    'Q': 'Queens',
    'BX': 'Bronx',
    'R': 'Staten Island'
}

@db.register.eval('borough_name')
def county_to_boro(county):
    return county_map[county]

In [ ]:
# Convert violation tome to 24 hour format (keeping
# only the hour value).

@db.register.eval('time_to_hour')
def time_to_hour24(value):
    if value.endswith('A'):
        return int(value[:2])
    else:
        h = int(value[:2])
        return h if h == 12 else h + 12

In [ ]:
# Show GUI spreadsheet view for a sample of 1000 rows from
# the dataset.

db.edit('street_cleaning_violations', n=1000, random_state=42)

In [ ]:
# Checkout the full dataset after applying all changes to it.

df = db.checkout('street_cleaning_violations', commit=True)

In [ ]:
# Filter invalid hour values and add counter for
# aggregation.

from openclean.function.eval.base import Col, Const
from openclean.function.eval.logic import And

df = stream(df)\
    .filter(And(Col('Violation Time') < 24, Col('Violation Time') >= 0))\
    .select(['Violation Time', 'Borough'])\
    .insert('Count', values=Const(1))\
    .to_df()

In [ ]:
# Use total counts of violations per borough to normalize
# the data.

from openclean.operator.collector.count import distinct
counts = distinct(df, 'Borough')

def normalize(boro, value):
    return value / counts[boro]

In [ ]:
# Plot violation probabilities by hour and borough using
# seaborn scatter plot.

import seaborn as sns
import matplotlib.pyplot as plt

from openclean.function.eval.base import Eval

sns.set_theme()
sns.set_context("talk", font_scale=1.)

# sns.set_style('ticks')
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)

viol = df.groupby(['Violation Time','Borough'])['Count'].sum().reset_index()
viol = stream(viol)\
    .update('Count', Eval(['Borough', 'Count'], normalize, is_unary=False))\
    .rename('Count', 'Probability')\
    .to_df()
sns.scatterplot(x='Violation Time', y='Probability', hue='Borough', palette='tab10', data=viol, ax=ax)